In [9]:
import chit

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
chat = chit.Chat.clone("foo2.json")
chat.model = "openrouter/anthropic/claude-3.7-sonnet"

In [11]:
chat.log("tree")

[S_]dfac3dae──[U*]203da29b──[A_]dab57316── (master)
            │             ├─[A_]2ad8645b── (retry_response)
            │             └─ (new_branch*)
            └─[U_]ecf21cad──[A_]af5de4e2──[U_]1aeaedd1──[A_]afacbdeb──[U_]f05b7bd4──[A_]8ee7390f── (edit_prompt)


In [12]:
import ast
import operator


def math_eval(expr: str) -> str | float:
    """
    Evaluate a simple mathematical expression. The supported operators are * (multiply), - (subtract),
    + (add), / (divide), and ** (power). If your expression was unable to be evaluated,
    the output will instead be "Invalid expression".

    Parameters
    ----------
    expr : str
        the expression to evaluate

    Returns
    -------
    float
        the result of the evaluation, or str "Invalid expression" if the expression is invalid
    """
    operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.USub: operator.neg,
        ast.Pow: operator.pow
    }
    def eval_node(node):
        if isinstance(node, ast.BinOp):
            if type(node.op) in operators:
                left = eval_node(node.left)
                right = eval_node(node.right)
                return operators[type(node.op)](left, right)
            else:
                raise ValueError(f"Unsupported operation {node.op}")
        elif isinstance(node, ast.Constant):
            return node.value
        else:
            raise ValueError(f"Unsupported type {type(node)}")

    try:
        # Parse the expression into an AST
        node = ast.parse(expr, mode='eval').body
        return str(eval_node(node))
    except Exception as e:
        return f"ERROR: {type(e)}"

In [13]:
chat.tools = [math_eval]
chat._recalc_tools()

In [14]:
chat.checkout(branch_name="master")
chat.commit("Now I have given you the ability to evaluate simple mathematical expressions. Try it out by multiplying 327*220!")

In [15]:
chat.commit() # call assistant

2025-03-03 00:45:46 - INFO - 1 tool calls pending; use .commit() to call one-by-one


I'd be happy to try out the math evaluation function with your expression! Let me calculate 327 multiplied by 220:


In [16]:
chat.commit() # run tool call
chat.commit() # call AI with this tool call result



The result of 327 × 220 = 71,940

Is there anything else you'd like me to calculate for you?
